In [ ]:
import os
import re
import json
import time
import pandas as pd
from openai import OpenAI
from textblob import TextBlob

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
BASE_PATH = "../data/silver"
GOLD_PATH = "../data/gold"
NUM_SAMPLES = 1000 # Número total de amostras desejadas
os.makedirs(GOLD_PATH, exist_ok=True)

try:
    df_listings = pd.read_csv(f"{BASE_PATH}/dim_listings.csv", quotechar='"', on_bad_lines='warn', low_memory=False)
    df_reviews = pd.read_csv(f"{BASE_PATH}/fact_reviews.csv", quotechar='"', on_bad_lines='warn')
    print(f"✅ Listings carregado: {df_listings.shape}")
    print(f"✅ Reviews carregado: {df_reviews.shape}")
except FileNotFoundError as e:
    print(f"❌ Erro: Arquivo não encontrado em {BASE_PATH}. Verifique se moveu os CSVs para a pasta data/silver!")
    raise e

✅ Listings carregado: (38670, 11)
✅ Reviews carregado: (264561, 5)


In [ ]:
def classify_review_gpt(comentario: str):
    print("Analisando review...")
    print("Comentário:", comentario)
    prompt = f"""
    Analise o seguinte review de Airbnb:

    "{comentario}"

    Formato obrigatório:
    {{
    "sentimento": "Positivo" | "Neutro" | "Negativo",
    "topico_principal": "Limpeza" | "Localização" | "Check-in" | "Outro",
    "sub_topico": "Conforto" | "Comunicação" | "Valor" | "Comodidades" | "Outro",
    "tom_de_urgencia": true | false
    }}
    """

    try:
        response = client.responses.create(
            model="gpt-4.1-mini",
            input=prompt,
        )
        raw_output = response.output[0].content[0].text.strip()
        print("RAW OUTPUT:", repr(raw_output))
        match = re.search(r"\{.*\}", raw_output, re.DOTALL)
    except Exception as e:
        error = str(e)

    if not match or 'error' in locals():
        return {
            "sentimento": None,
            "topico_principal": None,
            "sub_topico": None,
            "tom_de_urgencia": None,
            "erro": str(e)
        }
    
    json_text = match.group()
    return json.loads(json_text)


In [49]:
def classify_listing_gpt(text: str):
    prompt = f"""
    Atue como um especialista em Real Estate no Rio de Janeiro.
    Analise o título do anúncio: "{text}"
    
    Retorne APENAS um JSON válido com esta estrutura exata:
    {{
        "categoria_vibe": "Luxo | Econômico | Familiar | Romântico | Moderno | Padrão",
        "tipo_vista": "Mar | Natureza | Urbana | Sem Vista",
        "ponto_forte": "Uma frase curta de 3 palavras resumindo o destaque",
        "principal_caracteristica": "Unica palavra destacando o imóvel"
    }}
    """
    try:
        response = client.responses.create(
            model="gpt-4.1-mini",
            input=prompt,
        )
        raw_output = response.output[0].content[0].text.strip()
        # print("RAW OUTPUT:", repr(raw_output))
        match = re.search(r"\{.*\}", raw_output, re.DOTALL)
    except Exception as e:
        error = str(e)

    if not match or 'error' in locals():
        return {
            "categoria_vibe": None,
            "tipo_vista": None,
            "ponto_forte": None,
            "principal_caracteristica": None,
            "erro": str(e)
        }
    
    json_text = match.group()
    return json.loads(json_text)

In [ ]:
print("🎲 Iniciando Amostragem Inteligente de Reviews...")
df_reviews_head = df_reviews.head(NUM_SAMPLES/2).copy()
df_reviews_head['SAMPLE_TYPE'] = 'TOP_500'
df_reviews_remaining = df_reviews.iloc[NUM_SAMPLES/2:]
n_sample = min(NUM_SAMPLES/2, len(df_reviews_remaining))
df_reviews_random = df_reviews_remaining.sample(n=n_sample, random_state=42).copy()
df_reviews_random['SAMPLE_TYPE'] = 'RANDOM_500'
df_reviews_selection = pd.concat([df_reviews_head, df_reviews_random])

print("\n🏠 Iniciando Seleção de Listings Relacionados...")
related_listing_ids = df_reviews_selection['SK_LISTING'].unique()
df_listings_related = df_listings[df_listings['SK_LISTING'].isin(related_listing_ids)].copy()
df_listings_related['SAMPLE_ORIGIN'] = 'RELATED_TO_REVIEW'
count_related = len(df_listings_related)
print(f"   -> Encontrados {count_related} imóveis citados nos reviews selecionados.")
TARGET_LISTINGS = NUM_SAMPLES

if count_related < TARGET_LISTINGS:
    needed = TARGET_LISTINGS - count_related
    print(f"   -> Necessário completar com mais {needed} imóveis aleatórios.")
    df_listings_available = df_listings[~df_listings['SK_LISTING'].isin(related_listing_ids)]
    n_fill = min(needed, len(df_listings_available))
    
    df_listings_fill = df_listings_available.sample(n=n_fill, random_state=42).copy()
    df_listings_fill['SAMPLE_ORIGIN'] = 'RANDOM_FILL'
    df_listings_selection = pd.concat([df_listings_related, df_listings_fill])
    
else:
    print("   -> Quantidade de imóveis relacionados já supera {NUM_SAMPLES}. Limitando seleção.")
    df_listings_selection = df_listings_related.head(TARGET_LISTINGS)

print(f"✅ Seleção de Listings Concluída: {len(df_listings_selection)} registros.")
print(df_listings_selection['SAMPLE_ORIGIN'].value_counts())

🎲 Iniciando Amostragem Inteligente de Reviews...

🏠 Iniciando Seleção de Listings Relacionados...
   -> Encontrados 411 imóveis citados nos reviews selecionados.
   -> Necessário completar com mais 589 imóveis aleatórios.
✅ Seleção de Listings Concluída: 1000 registros.
SAMPLE_ORIGIN
RANDOM_FILL          589
RELATED_TO_REVIEW    411
Name: count, dtype: int64


In [ ]:

# # Reviews Analysis
print("🤖 Iniciando Análise de Reviews com GPT (Feature Engineering)...")
results = []
for index, row in df_reviews_selection.iterrows():
    results.append(classify_review_gpt(row['TXT_COMENTARIO']))
    # time.sleep(0.5)

df_features = pd.DataFrame(results, index=df_reviews_selection.index)
df_reviews_enriched = df_reviews_selection.drop('TXT_COMENTARIO', axis=1).join(df_features)
df_reviews_enriched.to_csv(f"{GOLD_PATH}/sample_reviews_enriched.csv", index=False)
# print("🤖 Iniciando Análise de Reviews com TextBlob (Feature Engineering)...")

🤖 Iniciando Análise de Reviews com GPT (Feature Engineering)...
Analisando review...
Comentário: El departamento es precioso y muy bien equipado! Superó lo publicado! Vera ha sido muy servicial y solidaria. No duden en alquilar este depto, excelentes condiciones y a 1 cuadra de la playa!
RAW OUTPUT: '{\n  "sentimento": "Positivo",\n  "topico_principal": "Localização",\n  "sub_topico": "Comodidades",\n  "tom_de_urgencia": false\n}'
Analisando review...
Comentário: Muy lindo apartamento, cómodo, moderno y bien ubicado. <br/>Muy buen trato tanto por parte de bernardo como por vera quien nos  recibió. 
RAW OUTPUT: '{\n  "sentimento": "Positivo",\n  "topico_principal": "Localização",\n  "sub_topico": "Conforto",\n  "tom_de_urgencia": false\n}'
Analisando review...
Comentário: The apartement is in a really great location in Copacabana. It is in walking distance to lots of restaurants and bars as well as the beach. Moreover, it is very spacious and equipped with everything needed for a nice s

In [ ]:
# Listings Analysis
print("🤖 Iniciando Análise de Listings com GPT (Feature Engineering)...")
results = []
for index, row in df_listings_selection.iterrows():
    results.append(classify_listing_gpt(row['NM_ANUNCIO']))
    # time.sleep(0.5)

df_features = pd.DataFrame(results, index=df_listings_selection.index)
df_listings_enriched = df_listings_selection.drop('NM_ANUNCIO', axis=1).join(df_features)
df_listings_enriched.to_csv(f"{GOLD_PATH}/sample_listings_enriched.csv", index=False)


🤖 Iniciando Análise de Listings com GPT (Feature Engineering)...
